In [ ]:
import pandas as pd

# Assuming filtered_df and month_hours are your DataFrames
# Replace 'filtered_df' and 'month_hours' with the actual names of your DataFrames

# Convert 'Contracts' to the same data type in both DataFrames if necessary
# filtered_df['Contracts'] = filtered_df['Contracts'].astype(the_desired_type)
# month_hours['Contracts'] = month_hours['Contracts'].astype(the_desired_type)

# Define a function to perform the dynamic lookup
def get_hours(row, month_hours_df):
    # Find the row in month_hours that matches the 'Contracts' value
    matching_row = month_hours_df[month_hours_df['Contracts'] == row['Contracts']]
    if not matching_row.empty:
        # Get the column name from the 'QBT' field of filtered_df
        column_name = row['QBT']
        # Retrieve the value from the matching row and column in month_hours
        return matching_row.iloc[0][column_name]
    return None

# Apply the function to create a new 'hours' column in filtered_df
filtered_df['hours'] = filtered_df.apply(get_hours, axis=1, args=(month_hours,))


In [ ]:
import pandas as pd

# Assuming df is your DataFrame and 'your_column' is the column you're basing your calculation on
# Replace 'new_column', 'calculation_column', and 'your_column' with the actual names of your columns

# Define the lambda function for the conditional calculation
df['new_column'] = df.apply(lambda row: (row['calculation_column'] * 100) if row['your_column'] == 'EXCH' 
                            else (row['calculation_column'] / 50) if row['your_column'] == 'Ex' 
                            else row['calculation_column'], axis=1)

# This will create a new column 'new_column' with the calculated values based on the condition


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Replace 'quantity', 'pnl', 'year' with the actual names of your columns if they are different

# Count the number of rows for each 'year' and 'quantity' combination
counts = df.groupby(['year', 'quantity']).size()

# Filter to get combinations where the count is exactly 12
valid_combinations = counts[counts == 12].index

# Filter the original DataFrame to include only rows with valid combinations
filtered_df = df[df.set_index(['year', 'quantity']).index.isin(valid_combinations)]

# Now, perform the grouping and summing of 'pnl', if filtered_df is not empty
if not filtered_df.empty:
    # Group by 'year' and 'quantity', then sum the 'pnl' values
    grouped_df = filtered_df.groupby(['year', 'quantity'], as_index=False)['pnl'].sum()

    # Rename the 'pnl' column dynamically based on the 'year' value
    # Assuming year value is consistent across all rows in the group
    year_value = grouped_df['year'].iloc[0]
    new_column_name = f'cal{year_value}'
    grouped_df.rename(columns={'pnl': new_column_name}, inplace=True)

    # grouped_df contains the summed 'pnl' values with the column renamed dynamically
else:
    print("No valid combinations with exactly 12 rows found.")


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Replace 'quantity', 'pnl', 'year' with the actual names of your columns if they are different

# Count the number of rows for each 'year' and 'quantity' combination
counts = df.groupby(['year', 'quantity']).size()

# Identify valid combinations where the count is exactly 12
valid_combinations = counts[counts == 12].index

# Separate the DataFrame into two parts: one that meets the criteria and one that doesn't
df_valid = df[df.set_index(['year', 'quantity']).index.isin(valid_combinations)]
df_invalid = df[~df.set_index(['year', 'quantity']).index.isin(valid_combinations)]

# Perform the grouping and summing of 'pnl' for the valid DataFrame
grouped_df = df_valid.groupby(['year', 'quantity'], as_index=False)['pnl'].sum()

# Rename the 'pnl' column in the grouped DataFrame
grouped_df.rename(columns={'pnl': 'cal'}, inplace=True)

# Append the grouped DataFrame to the invalid DataFrame
final_df = pd.concat([df_invalid, grouped_df])

# Sort the final DataFrame if needed, to mix the grouped and ungrouped rows appropriately
# final_df.sort_values(by=['year', 'quantity'], inplace=True)

# Now, final_df contains both the condensed rows and the original rows that didn't meet the criteria


In [ ]:
import pandas as pd
from datetime import datetime

# Assuming df is your DataFrame and 'contracts' is your column

# Mapping of month codes to month numbers
month_codes = {'F': 1, 'G': 2, 'H': 3, 'J': 4, 'K': 5, 'M': 6,
               'N': 7, 'Q': 8, 'U': 9, 'V': 10, 'X': 11, 'Z': 12}

# Function to convert contract code to datetime
def contract_to_datetime(contract):
    month = month_codes.get(contract[0])
    year = 2000 + int(contract[1:])  # Adjust year prefix '20' as needed
    return datetime(year, month, 1)

# Convert 'contracts' to datetime and sort
df['contract_date'] = df['contracts'].apply(contract_to_datetime)
df.sort_values(by='contract_date', ascending=False, inplace=True)
df.drop('contract_date', axis=1, inplace=True)  # Optional: remove the temporary column

# Now df is sorted with the most recent months on top
